In [6]:
import numpy as np
from specializeGraph import *
import warnings
warnings.filterwarnings('ignore')

In [16]:
def getFiedlers(A,base):
    """
    Returns the feedler eigenvalue before and after
    specialization
    """
    L = laplacian(A,randomWalk=0)
    eigs,vecs = np.linalg.eig(L)  
    fEig = np.sort(eigs)[1]
    
    sA = specializeGraph(A,base)
    sL = laplacian(sA,randomWalk=0)
    eigs,vecs = np.linalg.eig(L)  
    SfEig = np.sort(eigs)[1]
    return fEig,SfEig

def randomGraph(n,base=False,bSize=None):
    """
    Random Graph on n vertices with an optional 
    random base set of vertices
    """
    A = (np.random.rand(n,n)>np.random.rand())*1.
    for j in range(n): A[j,j] = 0
    nodes = list(range(n))
    if bSize is None:
        bSize = np.random.randint(1,high=n)
    base = list(np.random.choice(nodes,replace=False,size=bSize))
    
    if base:
        return A,base
    else:
        return A

### Density of Graphs that increase fielder eigenvalue

In [57]:
print("Density of Graphs that Increase Fiedler as n increases")
print("n\t Preserve \t Incr \t Decr")
for n in range(3,8):
    incr = 0
    decr = 0
    preserve = 0
    total = 10*n**3
    for i in range(total):
        A,base = randomGraph(n)
        L = laplacian(A,randomWalk=0)
        eigs,vecs = np.linalg.eig(L)  
        fEig = np.sort(eigs)[1]

        sA = specializeGraph(A,base)
        if sA.shape[0] > A.shape[0]:
            sL = laplacian(sA,randomWalk=0)
            eigs,vecs = np.linalg.eig(sL)  
            SfEig = np.sort(eigs)[1]
            fEig = np.round(rL,6)
            SfEig = np.round(SrL,6)

            if fEig < SfEig:
                incr += 1
            if fEig == SfEig:
                preserve += 1
            if fEig > SfEig:
                decr += 1
        else:
            total -= 1
            
    perPre = round(float(preserve)/(total),4)
    perIncr = round(float(incr)/(total),4)
    perDecr = round(float(decr)/(total),4)

    print("{}\t{}\t\t{}\t{}".format(n,perPre,perIncr,perDecr)) 

Density of Graphs that Increase Fiedler as n increases
n	 Preserve 	 Incr 	 Decr
3	0.0		1.0	0.0
4	0.0		1.0	0.0
5	0.0		1.0	0.0
6	0.0		1.0	0.0
7	0.0		1.0	0.0


### Changes in the Fiedler Eigenvector

In [ ]:
def splzTrackFiedler(A,Base):
    """
    Function to compute the specialization of a graph and keep track of
    Fiedler eigenvector. Base nodes and links between the base nodes 
    remain the same. The remaining nodes are specialized.
    
    Parameters
    ----------
    A (nxn ndarray): Adjacency matrix for a simple directed graph
    Base (list): base nodes (not to be specialized) zero indexed
    
    Returns
    -------
    S (pxp ndarray): Specialized adjacency matrix
    
    """
    if np.diag(A).sum() != 0:
        raise ValueError("Some vertices have self edges")
    
    n = A.shape[0]
    
    #Initialize the dictionary to keep track of nodes
    fiedler = dict()
    for l in range(n):
        fiedler[l] = []
    
    #Permute A so that the base nodes come first
    A = baseFirst(A,Base)
    bSize = len(Base)
    
    #Begin creating the block diagonal of specialized matrix
    B = A[:bSize,:bSize]
    diag = [B]
    links = []
    #Find connected components and compress graph
    smA,comp = compressGraph(A,bSize)   
    
    #Find all paths from a base node to a base node
    #through the connected components
    pressedPaths = findPathsToBase(smA,bSize)

    #For each compressed path find all combinations 
    #of nodes that pass through the associated components
    nNodes = bSize
    for Path in pressedPaths:
        compnts = [comp[c] for c in Path]
        paths = pathCombinations(A,compnts)
        #Select all components not in the base node
        compnToAdd = [A[c,:][:,c] for c in compnts[1:-1]]
        for p in paths:
            diag += compnToAdd
            links += linkAdder(p,nNodes,compnts)
            nNodes += sum(map(len,compnToAdd))
            
    S = block_diag(*diag)
    for l in links: S[l] = 1
    return S